# Preprocessing

## Libraries

In [4]:
import numpy as np
import pandas as pd
from json import dumps

from sklearn.model_selection import GroupKFold, GroupShuffleSplit, KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import make_scorer, fbeta_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_sample_weight

RANDOM_STATE = 1

## Definitions

In [5]:
trees = pd.read_csv('../data/raw/new_york_tree_census_2015.csv')
# print(trees.columns)

target = trees['health']
keep_rows = target.notna()

drop_cols = ['health','tree_id', 'block_id', 'created_at', 'stump_diam','status','spc_common','problems','address',\
             'zip_city','cb_num', 'borocode', 'cncldist', 'st_assem', 'st_senate', 'nta', 'boro_ct', 'state',\
             'latitude', 'longitude']

group = ['nta_name']
spatial_geo = ['x_sp','y_sp'] # encode numerical, standardize
spatial_fine = ['zipcode'] # encode categorical
spatial_coarse = ['boroname'] # encode categorical

## base ftrs, without spatial ##
numerical_ftrs = ['tree_dbh']
categorical_ftrs = ['curb_loc', 'spc_latin', 'user_type', 'root_stone',
                    'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
                    'brch_light', 'brch_shoe', 'brch_other']
ordinal_ftrs = ['steward','guards','sidewalk']
ordinal_cats = [['None','1or2','3or4','4orMore'],
                ['None','Harmful','Unsure','Helpful'],
                ['Damage','None','NoDamage']]

In [6]:
# check we have accounted for all columns
listed = set(drop_cols).union(group,spatial_geo,spatial_fine,spatial_coarse,\
                              numerical_ftrs,categorical_ftrs,ordinal_ftrs)
print(set(trees.columns).difference(listed) == set())

True


In [7]:
# handle ordinal ftrs to fill in na
for ftr in ordinal_ftrs:
    trees[ftr] = trees[ftr].fillna('None')

In [8]:
# define feature sets
drop_cols_set = drop_cols + group # + spatial_fine + spatial_coarse
categorical_ftrs_set = categorical_ftrs + spatial_fine + spatial_coarse
numerical_ftrs_set = numerical_ftrs + spatial_geo

# define a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories=ordinal_cats), ordinal_ftrs),
        ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'), categorical_ftrs_set),
        ('std', StandardScaler(), numerical_ftrs_set)])

## Sampling

In [9]:
SAMPLE_PROP = 0.01
y = target[keep_rows]
X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]

In [10]:
# hold out a test set by groups
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]

In [11]:
# subsample for cross validation
X_sub, _, y_sub, _, groups_sub, _ = train_test_split(
    X_train, y_train, groups_train,
    train_size=SAMPLE_PROP,
    stratify=y_train,
    random_state=RANDOM_STATE
)

# Cross Validation

## Functions

In [12]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", category=ConvergenceWarning)

In [13]:
random_states = [1,33,42,44,99]

In [14]:
from sklearn.base import BaseEstimator

def run_grid_search(random_states: list[int], model: BaseEstimator, X: pd.DataFrame | np.ndarray, y: pd.DataFrame | np.ndarray, groups: pd.DataFrame, \
                    param_grid: dict, n_jobs: int = -1, fit_params: dict = {}, pipe: Pipeline = None, add_rs: bool = True) \
                    -> tuple[list[np.float64],list[dict], list[BaseEstimator]]:
    scores, params, models = [], [], []
    for rs in random_states:
        gss = GroupShuffleSplit(n_splits=1,random_state=rs) # set to defualt 0.2 test
        pipe = make_pipeline(preprocessor,model) if pipe is None else pipe
        if add_rs:
            name = list(pipe.named_steps.keys())[-1] + '__random_state'
            param_grid[name] = [rs]
        grid = GridSearchCV(
            estimator=pipe, 
            param_grid=param_grid,
            scoring='f1_macro',
            cv=gss,
            verbose=False,
            n_jobs=n_jobs,
            refit=True,
            return_train_score=True
        )
        grid.fit(X, y, groups=groups, **fit_params)

        best_idx = grid.best_index_
        scores.append(grid.cv_results_['mean_test_score'][best_idx])
        params.append(grid.best_params_)
        models.append(grid.best_estimator_)

    return scores, params, models

In [15]:
def print_results(name: str, scores: list[np.float64], params: list[dict], \
                  models: list[BaseEstimator]) -> None:
    print(name.lower(), '----\n')
    indx = np.argmax(scores)

    print('best parameters:')
    print(dumps(params[indx],indent=4))

    print(f'score: {scores[indx]}')
    print(f'mean score: {np.mean(scores)}')
    print(f'std score:  {np.std(scores)}')

    return None

## Logistic Regression, Elastic Net

In [13]:
clf = LogisticRegression(penalty='elasticnet',solver='saga',max_iter=5_000)
param_grid = {
            'logisticregression__C': [0.01, 0.1, 1, 10], 
            'logisticregression__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
            'logisticregression__class_weight': ['balanced']
            }
scores_lr, params_lr, model_lr = \
    run_grid_search(random_states=random_states,
                    model=clf,
                    X=X_sub,
                    y=y_sub,
                    groups=groups_sub,
                    param_grid=param_grid,
                    n_jobs=-1)


In [14]:
print_results('logistic regression', scores_lr, params_lr, model_lr)

logistic regression ----

best parameters:
{
    "logisticregression__C": 1,
    "logisticregression__class_weight": "balanced",
    "logisticregression__l1_ratio": 0.3,
    "logisticregression__random_state": 42
}
score: 0.38616238364007627
mean score: 0.37118256858651005
std score:  0.011786660982739948


## SVM Classifier

In [15]:
from sklearn.svm import SVC
class_weights = {
    'Poor': np.sum(y.to_numpy()=='Poor') / len(y),
    'Fair': np.sum(y.to_numpy()=='Fair') / len(y),
    'Good': np.sum(y.to_numpy()=='Good') / len(y)
}
svc = SVC(class_weight=class_weights)
param_grid = {
            'svc__C': [0.01, 0.1, 1, 10],
            'svc__kernel': ['linear','poly','rbf','sigmoid']
            }
scores_svc, params_svc, model_svc = \
    run_grid_search(random_states=random_states,
                    model=svc,
                    X=X_sub,
                    y=y_sub,
                    groups=groups_sub,
                    param_grid=param_grid)


In [16]:
print_results('svm classifier', scores_svc, params_svc, model_svc)

svm classifier ----

best parameters:
{
    "svc__C": 10,
    "svc__kernel": "sigmoid",
    "svc__random_state": 99
}
score: 0.3198561828532294
mean score: 0.3115233981067871
std score:  0.006411327167816232


## KNN Classifier

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
param_grid = {
            'kneighborsclassifier__n_neighbors': [10,20,30,40,50],
            'kneighborsclassifier__weights': ['uniform','distance'],
            'kneighborsclassifier__n_jobs': [-1]
            }
scores_knn, params_knn, model_knn = \
    run_grid_search(random_states=random_states,
                    model=knn,
                    X=X_sub,
                    y=y_sub,
                    groups=groups_sub,
                    param_grid=param_grid,
                    n_jobs=1,
                    add_rs=False)

In [18]:
print_results('knn classifier', scores_knn, params_knn, model_knn)

knn classifier ----

best parameters:
{
    "kneighborsclassifier__n_jobs": -1,
    "kneighborsclassifier__n_neighbors": 10,
    "kneighborsclassifier__weights": "distance"
}
score: 0.32166812107523574
mean score: 0.315715937403959
std score:  0.005763146307245319


## XGBoost Classifier

In [16]:
# sample 5% and to get evaluations set
SAMPLE_PROP = 0.05

In [17]:
from copy import deepcopy
def xgb_subsample(prop: float, X: pd.DataFrame, y: pd.DataFrame, groups: pd.DataFrame)\
    -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, np.ndarray, list[pd.DataFrame]]:
    # set aside test set
    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
    train_idx, test_idx = next(gss.split(X, y, groups))
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]
    # subsample
    X_sub, _, y_sub, _, groups_sub, _ = train_test_split(
        X_train, y_train, groups_train,
        train_size=prop,
        stratify=y_train,
        random_state=RANDOM_STATE
    )
    # sample eval
    gss_eval = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
    sub_idx, eval_idx = next(gss_eval.split(X_sub, y_sub, groups_sub))
    X_sub, X_eval = X.iloc[sub_idx], X.iloc[eval_idx]
    y_sub, y_eval = y.iloc[sub_idx], y.iloc[eval_idx]
    groups_sub, groups_eval = groups.iloc[sub_idx], groups.iloc[eval_idx]
    # get weights
    sample_weights = compute_sample_weight(
        class_weight='balanced',
        y=y_sub
    )
    xg_pre = deepcopy(preprocessor)
    xg_pre.fit(X)
    X_sub_proc = xg_pre.transform(X_sub)
    X_eval_proc = xg_pre.transform(X_eval)
    return X_sub_proc, y_sub, groups_sub, sample_weights, X_eval_proc, y_eval,\
          X_test, y_test

In [18]:
y = target[keep_rows]
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))
X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]

In [19]:
X_sub_proc, y_sub, groups_sub, sample_weights, X_eval_proc, y_eval, _, _ =\
    xgb_subsample(prop=SAMPLE_PROP, X=X, y=y, groups=groups)

In [20]:
import xgboost as xgb
xgb = xgb.XGBClassifier(early_stopping_rounds=10,verbosity=0)
param_grid = {
            'xgbclassifier__learning_rate': [0.01, 0.1],
            'xgbclassifier__max_depth': [5, 10, 15, 20, 30, 50, 75, 100],
            'xgbclassifier__n_jobs': [-1]
            }
fit_params = {
    'xgbclassifier__sample_weight': sample_weights,
    'xgbclassifier__eval_set': [(X_eval_proc,y_eval)]
}
pipe = make_pipeline(xgb)
scores_xg, params_xg, model_xg = \
    run_grid_search(random_states=random_states,
                    model=xgb,
                    X=X_sub_proc,
                    y=y_sub,
                    groups=groups_sub,
                    param_grid=param_grid,
                    n_jobs=1,
                    fit_params=fit_params,
                    pipe=pipe)

[0]	validation_0-mlogloss:1.09768
[1]	validation_0-mlogloss:1.09675
[2]	validation_0-mlogloss:1.09581
[3]	validation_0-mlogloss:1.09489
[4]	validation_0-mlogloss:1.09400
[5]	validation_0-mlogloss:1.09310
[6]	validation_0-mlogloss:1.09224
[7]	validation_0-mlogloss:1.09136
[8]	validation_0-mlogloss:1.09049
[9]	validation_0-mlogloss:1.08965
[10]	validation_0-mlogloss:1.08881
[11]	validation_0-mlogloss:1.08797
[12]	validation_0-mlogloss:1.08717
[13]	validation_0-mlogloss:1.08636
[14]	validation_0-mlogloss:1.08556
[15]	validation_0-mlogloss:1.08478
[16]	validation_0-mlogloss:1.08399
[17]	validation_0-mlogloss:1.08319
[18]	validation_0-mlogloss:1.08244
[19]	validation_0-mlogloss:1.08164
[20]	validation_0-mlogloss:1.08091
[21]	validation_0-mlogloss:1.08013
[22]	validation_0-mlogloss:1.07939
[23]	validation_0-mlogloss:1.07863
[24]	validation_0-mlogloss:1.07790
[25]	validation_0-mlogloss:1.07716
[26]	validation_0-mlogloss:1.07643
[27]	validation_0-mlogloss:1.07574
[28]	validation_0-mlogloss:1.0

In [36]:
print_results('xgb classifier', scores_xg, params_xg, model_xg)
indx = np.argmax(scores_xg)
print(f"best trees: {model_xg[indx].named_steps['xgbclassifier'].best_iteration}")

xgb classifier ----

best parameters:
{
    "xgbclassifier__learning_rate": 0.01,
    "xgbclassifier__max_depth": 50,
    "xgbclassifier__n_jobs": -1,
    "xgbclassifier__random_state": 99
}
score: 0.402193383642752
mean score: 0.3946561484539992
std score:  0.004408163052166626
best trees: 99


# Final Model

`xgb classifier`->

* `learning_rate`: 0.01
* `max_depth`: 50
* `num_estimators`: 99
* `n_jobs`: -1

In [37]:
from xgboost import XGBClassifier
SAMPLE_PROP = 0.5
final = XGBClassifier(
    n_estimators = 99,
    max_depth = 50,
    learning_rate = 0.01,
    n_jobs = -1
)
y = target[keep_rows]
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))
X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]

X_sub_proc, y_sub, groups_sub, sample_weights, X_eval, y_eval, X_test, y_test =\
    xgb_subsample(prop=SAMPLE_PROP, X=X, y=y, groups=groups)
final.fit(X=X_sub_proc,
          y=y_sub,
          sample_weight=sample_weights,
          eval_set=[(X_eval, y_eval)],
          verbose=True)

[0]	validation_0-mlogloss:1.09266
[1]	validation_0-mlogloss:1.08669
[2]	validation_0-mlogloss:1.08083
[3]	validation_0-mlogloss:1.07509
[4]	validation_0-mlogloss:1.06945
[5]	validation_0-mlogloss:1.06388
[6]	validation_0-mlogloss:1.05843
[7]	validation_0-mlogloss:1.05304
[8]	validation_0-mlogloss:1.04775
[9]	validation_0-mlogloss:1.04252
[10]	validation_0-mlogloss:1.03740
[11]	validation_0-mlogloss:1.03234
[12]	validation_0-mlogloss:1.02736
[13]	validation_0-mlogloss:1.02245
[14]	validation_0-mlogloss:1.01760
[15]	validation_0-mlogloss:1.01282
[16]	validation_0-mlogloss:1.00810
[17]	validation_0-mlogloss:1.00348
[18]	validation_0-mlogloss:0.99890
[19]	validation_0-mlogloss:0.99437
[20]	validation_0-mlogloss:0.98991
[21]	validation_0-mlogloss:0.98551
[22]	validation_0-mlogloss:0.98114
[23]	validation_0-mlogloss:0.97683
[24]	validation_0-mlogloss:0.97258
[25]	validation_0-mlogloss:0.96839
[26]	validation_0-mlogloss:0.96425
[27]	validation_0-mlogloss:0.96017
[28]	validation_0-mlogloss:0.9

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [3]:
xg_pre = deepcopy(preprocessor)
xg_pre.fit(X)
X_test_proc = xg_pre.transform(X_test)
feature_names = xg_pre.get_feature_names_out()
df_test = pd.DataFrame(data=X_test_proc, columns=feature_names)
final_score = fbeta_score(y_test, final.predict(X_test_proc), beta=1, average='macro')

NameError: name 'deepcopy' is not defined

In [39]:
baseline_preds = y_test.copy()
baseline_preds.iloc[:, 0] = 1
baseline_score = fbeta_score(y_test, baseline_preds, beta=1, average='macro')

In [64]:
print(f'model score: {final_score}')
print(f'baseline:    {baseline_score}')

model score: 0.6073128841085816
baseline:    0.2990431574273948


In [51]:
# save the final model object
final.save_model("../models/xgboost_model_50.bin")
final.save_model("../models/xgboost_model_50.json")

/opt/miniconda3/envs/tree_census/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [03:22:43] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1727634913186/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


# Feature Interepretability

## SHAP

In [ ]:
import shap
explainer = shap.TreeExplainer(final)
shap_values = explainer.shap_values(X_test_proc)

In [ ]:
print(shap_values.shape)